In [1]:
import numpy as np

from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D,Conv2DTranspose,Flatten,Dense,Reshape
from keras.layers import LeakyReLU,BatchNormalization,Dropout,Lambda

from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import binary_crossentropy

import keras.backend as K

from keras.optimizers import Adam,RMSprop
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()



Keras Config

In [2]:
print("Keras Backend: ",K.backend())
print("Image Format : ",K.image_data_format())

K.set_image_data_format('channels_last')

Keras Backend:  tensorflow
Image Format :  channels_last


Building the Network

In [3]:
img_rows,img_cols,img_chns = 128,128,3

input_size = (img_rows,img_cols,img_chns)

filters = 64
kernel_size = 3
batch_size = 32
latent_dim = 2
intermed_dim = 128
num_epochs = 10

print('Original Image Shape = ',input_size)

Original Image Shape =  (128, 128, 3)


In [4]:
input_layer = Input(shape=input_size)
x = input_layer
#Conv Layer - 1
x = Conv2D(filters=img_chns, kernel_size=(2,2),padding='same')(x)
x = LeakyReLU()(x)
#Conv Layer - 2
x = Conv2D(filters=filters, kernel_size=(2,2), padding='same')(x)
x = LeakyReLU()(x)
#Conv Layer - 3
x = Conv2D(filters=filters, kernel_size=3, padding='same')(x)
x = LeakyReLU()(x)
#Conv Layer - 4
x = Conv2D(filters=filters, kernel_size=3, padding='same')(x)
x = LeakyReLU()(x)

shape_before_flatten = K.int_shape(x)
#Flatten Layer
x = Flatten()(x)
#Intermediate hidden layer
x = Dense(intermed_dim)(x)
x = LeakyReLU()(x)

mu = Dense(latent_dim)(x)
log_var = Dense(latent_dim)(x)

In [5]:
def sampling(args):
    mu,log_var = args
    epsilon = K.random_normal(shape=K.shape(mu), mean=0, stddev=1.0)
    return mu + K.exp(0.5 * log_var)*epsilon

latent_vec = Lambda(sampling,output_shape=(latent_dim,))([mu,log_var])

In [6]:
shape_before_flatten[1:]

(128, 128, 64)

In [7]:
temp_model_1 = Model(input_layer,(mu,log_var),name='encoder_mu_log_var')
temp_model_1.summary()

Model: "encoder_mu_log_var"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 3)  39          ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 128, 128, 3)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 64  832         ['leaky_re_lu[0]

In [8]:
hidden_deocoder = Dense(intermed_dim,activation='relu')
upsampled_decoder = Dense(filters * 128 * 128, activation='relu')

In [9]:
output_shape = (batch_size,128,128,filters)
print("The output shape 1 = ", output_shape)

The output shape 1 =  (32, 128, 128, 64)


In [10]:
decoder_reshape = Reshape(output_shape[1:])
deconv_1 = Conv2DTranspose(filters, kernel_size=kernel_size, padding='same',activation='relu')
deconv_2 = Conv2DTranspose(filters,kernel_size=kernel_size,padding='same',activation='relu')

deconv_3 = Conv2DTranspose(filters, kernel_size=(3,3),strides=(2,2), padding='valid', activation='relu')
deconv_mean_squash = Conv2D(img_chns,kernel_size=2,padding='valid',activation='sigmoid')

output_shape = (batch_size,256,256,filters)

y = hidden_deocoder(latent_vec)
y = upsampled_decoder(y)
y = decoder_reshape(y)
y = deconv_1(y)
y = deconv_2(y)
y = deconv_3(y)
y = deconv_mean_squash(y)

In [11]:
VAE = Model(input_layer,y)
VAE.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 3)  39          ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 128, 128, 3)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 64  832         ['leaky_re_lu[0][0]']        

In [12]:
def vae_loss(y,y_hat):
    rl_loss = img_rows*img_cols*binary_crossentropy(K.flatten(y),K.flatten(y_hat))
    kl_loss = -0.5 * K.sum(1+log_var - K.square(mu) - K.exp(log_var),axis=-1)
    net_loss = K.mean(rl_loss+kl_loss)
    return net_loss

In [13]:
VAE.compile(optimizer='rmsprop',loss=vae_loss)

In [36]:
data_path = 'G:\Generative-deep-learning\Projects\celeb-face-vae\celeba-dataset\img_align_celeba'
img_gen = ImageDataGenerator(rescale=1./255,data_format='channels_last')
flow = img_gen.flow_from_directory(data_path, target_size=input_size[:2], batch_size=batch_size,shuffle=True,
                                    subset='training')

Found 202599 images belonging to 1 classes.


In [15]:
VAE.fit(flow,steps_per_epoch=202599//batch_size,epochs=5)

Epoch 1/5


TypeError: in user code:

    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "d:\condaa\envs\tflow2.9\lib\site-packages\keras\engine\compile_utils.py", line 239, in __call__
        self._loss_metric.update_state(
    File "d:\condaa\envs\tflow2.9\lib